In [9]:
# Import packages
import os
import random
from PIL import Image
from torch import nn
import torch 
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

from src.config import PROCESSED_DATA_DIR, RAW_DATA_DIR
image_path = RAW_DATA_DIR
train_dir = image_path / 'train'
test_dir = image_path / 'test'

In [10]:
import json

device ='cuda' if torch.cuda.is_available() else 'cpu'

config_path = 'params.json'
with open(config_path, 'r') as config_file:
    config = json.load(config_file)
print(json.dumps(config, indent=2))

{
  "model_params": {
    "learning_rate": 0.001,
    "batch_size": 64,
    "num_epochs": 25,
    "dropout_rate": 0.3,
    "optimizer": "adam",
    "loss_function": "categorical_crossentropy",
    "metrics": [
      "accuracy"
    ],
    "output_shape": 3
  }
}


In [11]:
# Load hyperparams from params.json

learning_rate = config['model_params']['learning_rate']
batch_size = config['model_params']['batch_size']
num_epochs = config['model_params']['num_epochs']
dropout_rate = config['model_params']['dropout_rate']
optimizer = config['model_params']['optimizer']
loss_function = config['model_params']['loss_function']
metrics = config['model_params']['metrics']
output_shape = config['model_params']['output_shape']

In [12]:
# Prepare data into dataloader
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
])

train_data = ImageFolder(train_dir, transform=transform)
test_data = ImageFolder(test_dir, transform=transform)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [31]:
class HHP01(nn.Module):
    def __init__(self, input_shape: int,
                 hidden_units: int,
                 output_shape: int):
        super(HHP01, self).__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units*53*53, out_features=output_shape)
        )
        
    def forward(self, x):
        # x = self.conv_block1(x)
        # print(x.shape)
        # x = self.conv_block2(x)
        # print(x.shape)
        # x = self.classifier(x)
        # print(x.shape)
        # return x
        return self.classifier(self.conv_block2(self.conv_block1(x)))

In [32]:
model = HHP01(input_shape=3, hidden_units=10, output_shape=output_shape).to(device)
model

HHP01(
  (conv_block1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=28090, out_features=3, bias=True)
  )
)

In [33]:
image_batch, label_batch = next(iter(train_loader))
image_batch.shape, label_batch.shape

(torch.Size([64, 3, 224, 224]), torch.Size([64]))

In [34]:
model(image_batch.to(device))

tensor([[-0.0072,  0.0450, -0.0343],
        [-0.0018,  0.0472, -0.0380],
        [-0.0064,  0.0504, -0.0319],
        [-0.0003,  0.0459, -0.0383],
        [-0.0059,  0.0510, -0.0324],
        [-0.0002,  0.0469, -0.0318],
        [-0.0043,  0.0475, -0.0335],
        [-0.0044,  0.0438, -0.0396],
        [ 0.0016,  0.0399, -0.0409],
        [-0.0072,  0.0410, -0.0318],
        [-0.0040,  0.0470, -0.0395],
        [-0.0062,  0.0493, -0.0357],
        [ 0.0008,  0.0460, -0.0392],
        [-0.0049,  0.0503, -0.0334],
        [-0.0058,  0.0505, -0.0313],
        [-0.0063,  0.0511, -0.0374],
        [-0.0062,  0.0477, -0.0384],
        [-0.0034,  0.0455, -0.0342],
        [-0.0066,  0.0394, -0.0365],
        [-0.0042,  0.0471, -0.0350],
        [-0.0040,  0.0503, -0.0433],
        [-0.0055,  0.0429, -0.0398],
        [-0.0031,  0.0488, -0.0339],
        [-0.0044,  0.0483, -0.0386],
        [-0.0003,  0.0447, -0.0417],
        [-0.0104,  0.0536, -0.0320],
        [-0.0114,  0.0469, -0.0358],
 

In [35]:
import torchinfo
torchinfo.summary(model, input_size=(1, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
HHP01                                    [1, 3]                    --
├─Sequential: 1-1                        [1, 10, 110, 110]         --
│    └─Conv2d: 2-1                       [1, 10, 222, 222]         280
│    └─ReLU: 2-2                         [1, 10, 222, 222]         --
│    └─Conv2d: 2-3                       [1, 10, 220, 220]         910
│    └─MaxPool2d: 2-4                    [1, 10, 110, 110]         --
├─Sequential: 1-2                        [1, 10, 53, 53]           --
│    └─Conv2d: 2-5                       [1, 10, 108, 108]         910
│    └─ReLU: 2-6                         [1, 10, 108, 108]         --
│    └─Conv2d: 2-7                       [1, 10, 106, 106]         910
│    └─MaxPool2d: 2-8                    [1, 10, 53, 53]           --
├─Sequential: 1-3                        [1, 3]                    --
│    └─Flatten: 2-9                      [1, 28090]                --
│    └─Line

In [39]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

def train_model(num_epochs):
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct / total
        print(f'Epoch {epoch + 1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')

# Call the training function
train_model(10)

Epoch 1, Loss: 1.0986, Accuracy: 34.67%
Epoch 2, Loss: 1.0956, Accuracy: 35.73%
Epoch 3, Loss: 1.0883, Accuracy: 40.58%
Epoch 4, Loss: 1.0745, Accuracy: 42.40%
Epoch 5, Loss: 1.0470, Accuracy: 46.93%
Epoch 6, Loss: 1.0280, Accuracy: 47.33%
Epoch 7, Loss: 1.0042, Accuracy: 49.69%
Epoch 8, Loss: 0.9870, Accuracy: 51.42%
Epoch 9, Loss: 0.9710, Accuracy: 53.16%
Epoch 10, Loss: 0.9610, Accuracy: 53.42%
